In [1]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd
import re
import math
import os
import ast

In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, '크롤링')
original_table_path = os.path.join(data_dir, r'원본(크롤링)\original_NE.csv')
dron_park_path = os.path.join(data_dir, r'원본(크롤링)\df_dron_park.csv')
original_table = pd.read_csv(original_table_path, encoding='utf-8')
dron_park = pd.read_csv(dron_park_path, encoding='utf-8')
# original_table = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\original_NE.csv", encoding='utf-8')
# dron_park = pd.read_csv(r"C:\Users\user\Desktop\python 정리집\웹크롤링 팀프로젝트\df_dron_park.csv", encoding='utf-8')
print(original_table)
print(dron_park)

      구역 코드   위치                                               수평범위  \
0      UA 2  구성산  Circle with radius of 1.8 km (1.0 NM) centered...   
1      UA 3   약산  Circle with radius of 0.7 km (0.4 NM) centered...   
2      UA 4  봉화산  Circle with radius of 4.0 km (2.2 NM) centered...   
3      UA 5  덕두산  Circle with radius of 4.5 km (2.4 NM) centered...   
4      UA 6   금산  Circle with radius of 2.1 km (1.1 NM) centered...   
5      UA 7   홍산  Circle with radius of 1.2 km (0.7 NM) centered...   
6      UA 9   양평  373010N 1272300E - 373010N 1273200E - 372700N ...   
7     UA 10   고창  Circle with radius of 4.0 km (2.2 NM) centered...   
8     UA 14   공주  363038N 1270033E - 363002N 1270713E - 362604N ...   
9     UA 19  시화호  371751N 1264215E - 371724N 1265000E - 371430N ...   
10    UA 21  방장산  Circle with radius of 3.0 km (1.6 NM) centered...   
11    UA 24   구좌  Circle with radius of 2.8 km (1.5 NM) centered...   
12    UA 25   하동  350147N 1274325E - 350145N 1274741E - 345915N ...   
13    

In [3]:
def dms_to_dd(dms_str):
    m = re.match(r"(\d{2,3})(\d{2})(\d{2})([NSEW])", dms_str.replace(" ", ""))
    if not m:
        return None
    deg, minute, second, direction = m.groups()
    dd = int(deg) + int(minute) / 60 + int(second) / 3600
    if direction in ["S", "W"]:
        dd *= -1
    return dd

In [4]:
def extract_ne_data(text):
    matches = re.findall(r'(\d{6,7}N)\s*(\d{6,7}E)', text)
    result = []
    for lat_str, lon_str in matches:
        lat = dms_to_dd(lat_str)
        lon = dms_to_dd(lon_str)
        result.append((lat, lon))
    return result if result else None

def order_coords(coords):
    if not coords or len(coords) <= 1:
        return coords
    cx = sum(pt[0] for pt in coords) / len(coords)
    cy = sum(pt[1] for pt in coords) / len(coords)
    return sorted(coords, key=lambda pt: math.atan2(pt[1]-cy, pt[0]-cx))

In [5]:
def build_ne_table():
    df = original_table
    df['NE'] = None
    for idx, row in df.iterrows():
        text = row['수평범위']
        if text and "Circle" not in text:
            ne_coords = extract_ne_data(text)
            if ne_coords and len(ne_coords) > 1:
                ne_coords = order_coords(ne_coords)
            df.at[idx, 'NE'] = ne_coords
    return df[df['NE'].notnull()][["구역 코드", "위치", "NE"]].reset_index(drop=True)

In [6]:
ne_table = build_ne_table()
print(ne_table)

      구역 코드   위치                                                 NE
0      UA 9   양평  [(37.45, 127.38333333333334), (37.502777777777...
1     UA 14   공주  [(36.45805555555556, 126.96388888888889), (36....
2     UA 19  시화호  [(37.212500000000006, 126.67472222222223), (37...
3     UA 25   하동  [(34.98777777777778, 127.72333333333333), (35....
4     UA 26  장암산  [(37.36972222222222, 128.39194444444445), (37....
5   * UA 31   청라  [(37.5625, 126.62666666666667), (37.565, 126.6...
6   * UA 33  병천천  [(36.647777777777776, 127.34972222222221), (36...
7   * UA 34  미호천  [(36.61388888888889, 127.3425), (36.6194444444...
8   * UA 35   김해  [(35.34527777777778, 128.8063888888889), (35.3...
9   * UA 36   밀양  [(35.46388888888889, 128.77416666666667), (35....
10  * UA 37   창원  [(35.37027777777778, 128.64888888888888), (35....
11  * UA 38   울주  [(35.52361111111111, 129.1627777777778), (35.5...
12  * UA 39   김제  [(35.91083333333333, 126.89194444444445), (35....
13  * UA 41   대전  [(36.45194444444445, 127.385),

In [7]:
def dms_to_dd(dms_str):
    m = re.match(r"(\d{2,3})(\d{2})(\d{2})([NSEW])", dms_str.replace(" ", ""))
    if not m:
        return None
    deg, minute, second, direction = m.groups()
    dd = int(deg) + int(minute) / 60 + int(second) / 3600
    if direction in ["S", "W"]:
        dd *= -1
    return dd

In [8]:
def extract_circle_data(text, region):
    m = re.search(
        r'Circle with radius of ([\d.]+)\s*(km|m).*?centered on\s*([\d]{6,7}[NSEW])\s*([\d]{6,7}[NSEW])',
        text
    )
    if m:
        radius = float(m.group(1))
        unit = m.group(2).lower()
        if unit == "m":
            radius /= 1000
        lat_str = m.group(3)
        lon_str = m.group(4)
        # 지역명이 "광주"인 경우에만 경도 교정 (예: 128... → 126...)
        if region and "광주" in region:
            if lon_str.startswith("128"):
                lon_str = "126" + lon_str[3:]
        return radius, dms_to_dd(lat_str), dms_to_dd(lon_str)
    return None, None, None

In [9]:
# "드론공원"까지의 부분만 남기는 함수
def extract_location_name(region):
    if pd.isnull(region):
        return region
    if "드론공원" in region:
        return region.split("드론공원")[0] + "드론공원"
    return region

In [10]:
# 좌표 컬럼이 문자열 형태로 저장되어 있다면 ast.literal_eval로 리스트로 변환
dron_park["좌표"] = dron_park["좌표"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# '지역' 컬럼의 값을 '위치' 컬럼으로 재설정 (드론공원까지의 부분만 남김)
dron_park["위치"] = dron_park["지역"].apply(extract_location_name)

# 신규 데이터를 NE와 Circle 데이터로 분리
# NE: 좌표 목록의 길이가 2 이상인 경우 (예: 대전 금강변 드론공원)
df_NE_new = dron_park[dron_park["좌표"].apply(lambda x: isinstance(x, list) and len(x) > 1)].copy()
df_NE_new["NE"] = df_NE_new["좌표"].apply(lambda x: str(x))
df_NE_new['구역 코드'] = ''
df_NE_new = df_NE_new[["구역 코드", "위치", "NE"]]
print(df_NE_new)

  구역 코드           위치                                                 NE
0        대전 금강변 드론공원  [(36.465, 127.39055555555557), (36.46583333333...


In [11]:
final_ne = pd.concat([ne_table, df_NE_new])

In [12]:
final_ne

,구역 코드,위치,NE
0,UA 9,양평,"[(37.45, 127.38333333333334), (37.502777777777..."
1,UA 14,공주,"[(36.45805555555556, 126.96388888888889), (36...."
2,UA 19,시화호,"[(37.212500000000006, 126.67472222222223), (37..."
3,UA 25,하동,"[(34.98777777777778, 127.72333333333333), (35...."
4,UA 26,장암산,"[(37.36972222222222, 128.39194444444445), (37...."
5,* UA 31,청라,"[(37.5625, 126.62666666666667), (37.565, 126.6..."
6,* UA 33,병천천,"[(36.647777777777776, 127.34972222222221), (36..."
7,* UA 34,미호천,"[(36.61388888888889, 127.3425), (36.6194444444..."
8,* UA 35,김해,"[(35.34527777777778, 128.8063888888889), (35.3..."
9,* UA 36,밀양,"[(35.46388888888889, 128.77416666666667), (35...."


In [14]:
# 위치 변경 매핑을 정의합니다.
location_map = {
    '공주': '세종(공주)',
    '시화호': '수원(시화호)',
    '하동': '광양(하동)',
    '장암산': '영월(장암산)',
    '청라': '인천(청라)',
    '병천천': '청주(병천천)',
    '미호천': '청주(미호천)',
    '울주': '울산(울주)',
    '김제': '군산(김제)',
    '대전 금강변 드론공원': '대전(대전 금강변 드론공원)'
}

# '위치' 컬럼에 있는 값들을 매핑된 값으로 바꿔줍니다.
final_ne['위치'] = final_ne['위치'].map(location_map).fillna(final_ne['위치'])

final_ne["지역"] = final_ne["위치"].str[:2]

print(final_ne)

      구역 코드               위치  \
0      UA 9               양평   
1     UA 14           세종(공주)   
2     UA 19          수원(시화호)   
3     UA 25           광양(하동)   
4     UA 26          영월(장암산)   
5   * UA 31           인천(청라)   
6   * UA 33          청주(병천천)   
7   * UA 34          청주(미호천)   
8   * UA 35               김해   
9   * UA 36               밀양   
10  * UA 37               창원   
11  * UA 38           울산(울주)   
12  * UA 39           군산(김제)   
13  * UA 41               대전   
14  **UA 43               영월   
0            대전(대전 금강변 드론공원)   

                                                   NE  지역  
0   [(37.45, 127.38333333333334), (37.502777777777...  양평  
1   [(36.45805555555556, 126.96388888888889), (36....  세종  
2   [(37.212500000000006, 126.67472222222223), (37...  수원  
3   [(34.98777777777778, 127.72333333333333), (35....  광양  
4   [(37.36972222222222, 128.39194444444445), (37....  영월  
5   [(37.5625, 126.62666666666667), (37.565, 126.6...  인천  
6   [(36.647777777777776, 127.34972

In [ ]:
data_dir = os.path.join(parent_dir, '데이터')
original_table_path = os.path.join(data_dir, r'전처리\NE_table.csv')
circle_table.to_csv(original_table_path, encoding='cp949', index=False)